In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


In [2]:
#Load the Saved model
loaded_model = xgb.Booster()
loaded_model.load_model('LeoPayu_BBILAP_weightVintage_15_in_24mob.json')
print("Model Loaded Successfully")

Model Loaded Successfully


In [3]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [4]:
combined_data = model_data.copy()

In [5]:
#Make sure select correct features or not
loaded_model.feature_names

['Num_Enq_all_loans_last_9M',
 'Num_of_PERSONAL LOAN_opened_last9M',
 'num_PERSONAL LOAN_enq_last_36M',
 'num_CONSUMER LOAN_enq_last_36M',
 'num_CREDIT CARD_enq_last_18M',
 'Total_outstanding_Balance_CREDIT CARD',
 'highest_sanction_amount_credit_cards',
 'Age_of_oldest_open_account',
 'num_PL_LE_1_Lac_opened_in_last_12M',
 'num_PL_LE_1_Lac_with_no_delinquency',
 'Total_Sanction_amt_PERSONAL LOAN_Opened_last_6M',
 'Utilization_PERSONAL LOAN_opened_last_9M',
 'Utilization_all',
 'num_of_loans_le_1lac_non_credit',
 'percentage_ever_delinquent',
 'months_since_most_recent_delinquency_installment',
 'percentage_utilization_gt_75',
 'No of enq in past 90days',
 'Num_Enq_non_BANK_last_12M',
 'Num_of_installment_loan_opened',
 'num_debits_gt_10000_M3',
 'num_credit_gt_salary',
 'upi_trans_count',
 'ratio_balance_after_7_days_max_credit_M3',
 'cash_withdrawal_sum',
 'num_cash_withdrawal_last_1_months',
 'bounced_transactions_count',
 'min_balance_last_3_months',
 'days_balance_lt_5000_M2',
 'd

In [6]:
combined_data['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
NaN    354961
0.0    109935
1.0     13432
Name: count, dtype: int64

In [7]:
combined_data['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
NaN    354961
0.0    109935
1.0     13432
Name: count, dtype: int64

In [8]:
combined_df= combined_data.copy()

In [9]:
df1 = combined_df.dropna(subset=['60_in_15mob'])

In [10]:
df1.shape

(123367, 2831)

In [11]:
df1['user_segment'].value_counts()

user_segment
OFFUS    90704
ONUS     32663
Name: count, dtype: int64

In [12]:
df = df1.copy()

In [13]:
df1.shape

(123367, 2831)

In [14]:
df1['unique_reference_number'].nunique()

123367

In [15]:
#change the month-year (Various Vintage)
df = df[df['month_year']=='2023-04']

In [16]:
df.shape

(6050, 2831)

In [17]:
data = df.copy()

In [75]:
import numpy as np
data.replace([np.inf, -np.inf], np.nan, inplace=True)

/tmp/ipykernel_2855962/499432767.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace([np.inf, -np.inf], np.nan, inplace=True)


In [18]:
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2023-03')]
oot_df1= data[(data['month_year']>='2023-04') & (data['month_year']<='2023-06')]

In [19]:
model_df1.shape, oot_df1.shape

((0, 2831), (6050, 2831))

In [20]:
features_list = ['Num_Enq_all_loans_last_9M',
'Num_of_PERSONAL LOAN_opened_last9M',
'num_PERSONAL LOAN_enq_last_36M',
'num_CONSUMER LOAN_enq_last_36M',
'num_CREDIT CARD_enq_last_18M',
'Total_outstanding_Balance_CREDIT CARD',
'highest_sanction_amount_credit_cards',
'Age_of_oldest_open_account',
'num_PL_LE_1_Lac_opened_in_last_12M',
'num_PL_LE_1_Lac_with_no_delinquency',
'Total_Sanction_amt_PERSONAL LOAN_Opened_last_6M',
'Utilization_PERSONAL LOAN_opened_last_9M',
'Utilization_all',
'num_of_loans_le_1lac_non_credit',
                 
# Reconcilated Features
'percentage_ever_delinquent',
'months_since_most_recent_delinquency_installment',
'percentage_utilization_gt_75',
'No of enq in past 90days',
'Num_Enq_non_BANK_last_12M',
'Num_of_installment_loan_opened',
'num_debits_gt_10000_M3',
'num_credit_gt_salary',
'upi_trans_count',
'ratio_balance_after_7_days_max_credit_M3',
'cash_withdrawal_sum',
'num_cash_withdrawal_last_1_months',
'bounced_transactions_count',
'min_balance_last_3_months',
'days_balance_lt_5000_M2',
'days_balance_lt_10000_M3',
'avg_num_credits_last_2_months',
'Max_Credit_Avg_M1_M2_M3',
'ratio_3th_day_balance_to_max_credit_M2',
'ratio_active_loan_vs_salary',
'num_self_trans_last_3_months',


'deviation_from_mean_income#last_30_Days',
'age',



'distinct_merchant',
'MEAN(raw.amount WHERE device_os_category = Windows)#ever',
'days_on_pg',
'NUM_UNIQUE(pg_success_raw_data.merchant_id WHERE revised_merchant_category = marketplace)',
'amt_first_android_txn', 
                 


'dm206s',
'g406s',
'balmag01',
'aggs907',

              
'month_year',
'60_in_15mob']

In [21]:
model_df = model_df1[features_list]

In [22]:
oot_df = oot_df1[features_list]

In [23]:
model_df.shape, oot_df.shape

((0, 48), (6050, 48))

In [24]:
# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '60_in_15mob']) # Adjust 'target' to your actual target column name
y_model = model_df['60_in_15mob']

In [25]:
X_oot = oot_df.drop(columns=['month_year', '60_in_15mob'])
y_oot = oot_df['60_in_15mob']

In [26]:
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3043639/3109673050.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)


In [27]:
for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3043639/1979277635.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_oot[i].fillna(-9999,inplace=True)


In [28]:
X_oot.shape,y_oot.shape

((6050, 46), (6050,))

In [29]:
# # Create DMatrix for OOT predictions
# if records available on OOT split use this cell else use below cell
data_x_pred = xgb.DMatrix(X_oot)
X_oot['60_in_15mob'] = y_oot
X_oot['y_pred'] = loaded_model.predict(data_x_pred)

In [ ]:
# #if records available on model split use this cell else use above cell
# data_x_pred = xgb.DMatrix(X_model)
# X_model['60_in_15mob'] = y_model
# X_model['month_year'] = model_df.loc[model_df.index,'month_year'].values

# X_model['y_pred'] = loaded_model.predict(data_x_pred)

In [30]:

summary,auc, gini, ks = decile_analysis(X_oot, 'y_pred', '60_in_15mob')  #if records available on OOT split use this cell else use below cell
# summary,auc, gini, ks = decile_analysis(X_model, 'y_pred', '60_in_15mob') #if records available on model split use this cell else use above cell


In [31]:
print(f"AUC: {auc:.2f}, Gini: {gini:.2f}, KS: {ks:.2f}")

AUC: 0.68, Gini: 0.37, KS: 0.29


In [32]:
summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.236364         0.141097        0.224019        0.116838    605   
9          0.163636         0.104492        0.116838        0.094269    605   
8          0.152066         0.087117        0.094262        0.080143    605   
7          0.122314         0.074494        0.080101        0.069770    605   
6          0.114050         0.066929        0.069765        0.064583    605   
5          0.057851         0.062529        0.064565        0.060314    605   
4          0.080992         0.057677        0.060311        0.055072    605   
3          0.049587         0.053315        0.055070        0.051704    605   
2          0.041322         0.050144        0.051703        0.048521    605   
1          0.028099         0.045979        0.048518        0.041052    605   

        label_good  label_bad  
Decile                         
10             462        143  
9              506         99  
8              513         92  
7              531         74  
6              536         69  
5              570         35  
4              556         49  
3              575         30  
2              580         25  
1              588         17

In [33]:
feature_importance = loaded_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                              Feature  Importance
35                                         days_on_pg       111.0
39                                              g406s        92.0
17                          Num_Enq_non_BANK_last_12M        71.0
23                         bounced_transactions_count        62.0
0                           Num_Enq_all_loans_last_9M        62.0
12                                    Utilization_all        44.0
1                  Num_of_PERSONAL LOAN_opened_last9M        38.0
34  MEAN(raw.amount WHERE device_os_category = Win...        36.0
20                                    upi_trans_count        33.0
7                          Age_of_oldest_open_account        31.0
31            deviation_from_mean_income#last_30_Days        24.0
19                               num_credit_gt_salary        23.0
27                      avg_num_credits_last_2_months        20.0
32                                                age        20.0
3         

In [34]:
feature_importance_df

Feature  Importance
35                                         days_on_pg       111.0
39                                              g406s        92.0
17                          Num_Enq_non_BANK_last_12M        71.0
23                         bounced_transactions_count        62.0
0                           Num_Enq_all_loans_last_9M        62.0
12                                    Utilization_all        44.0
1                  Num_of_PERSONAL LOAN_opened_last9M        38.0
34  MEAN(raw.amount WHERE device_os_category = Win...        36.0
20                                    upi_trans_count        33.0
7                          Age_of_oldest_open_account        31.0
31            deviation_from_mean_income#last_30_Days        24.0
19                               num_credit_gt_salary        23.0
27                      avg_num_credits_last_2_months        20.0
32                                                age        20.0
3                      num_CONSUMER LOAN_enq_last_36M        19.0
11           Utilization_PERSONAL LOAN_opened_last_9M        18.0
13                         percentage_ever_delinquent        16.0
37                              amt_first_android_txn        15.0
33                                  distinct_merchant        14.0
36  NUM_UNIQUE(pg_success_raw_data.merchant_id WHE...        13.0
25                            days_balance_lt_5000_M2        12.0
15                       percentage_utilization_gt_75        11.0
16                           No of enq in past 90days        11.0
40                                           balmag01        10.0
14   months_since_most_recent_delinquency_installment        10.0
10    Total_Sanction_amt_PERSONAL LOAN_Opened_last_6M         9.0
26                           days_balance_lt_10000_M3         9.0
22                                cash_withdrawal_sum         9.0
9                 num_PL_LE_1_Lac_with_no_delinquency         8.0
8                  num_PL_LE_1_Lac_opened_in_last_12M         7.0
4                        num_CREDIT CARD_enq_last_18M         6.0
24                          min_balance_last_3_months         5.0
2                      num_PERSONAL LOAN_enq_last_36M         5.0
6                highest_sanction_amount_credit_cards         5.0
29             ratio_3th_day_balance_to_max_credit_M2         5.0
28                            Max_Credit_Avg_M1_M2_M3         4.0
38                                             dm206s         3.0
5               Total_outstanding_Balance_CREDIT CARD         2.0
41                                            aggs907         2.0
18                             num_debits_gt_10000_M3         2.0
21           ratio_balance_after_7_days_max_credit_M3         1.0
30                        ratio_active_loan_vs_salary         1.0